#### Chapter 2 : Introduction to Design Patterns and Neural Networks

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loading the dataset
fashionData = tf.keras.datasets.fashion_mnist
# Get the train and test sets along with the respective labels
(train_img, train_labels), (test_img, test_labels) = fashionData.load_data()

In [ ]:
print('Shape of training data',train_img.shape)
print('Shape of test data',test_img.shape)


In [ ]:
import random

In [ ]:
# Take a random index
index = random.randint(0,60000)
print("Class of the product",train_labels[index])
# Extract the imge from the training set
img = train_img[index,:,:]
print("Shape of the image",img.shape)


In [ ]:
# Plotting the image
plt.imshow(img)
plt.colorbar()
# Showing the image
plt.show()

In [ ]:
# Getting the pixel values of the image
img

In [ ]:
# Scale the image data
train_img = train_img / 255.0
test_img = test_img / 255.0

#### Classification model for Fashion MNIST

In [ ]:
# Import the classes for building the model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,ReLU,Flatten,Softmax,Dropout
%load_ext tensorboard

In [ ]:
# Build the model using the Sequential class
model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(128))
model.add(ReLU())
model.add(Dense(128))
model.add(ReLU())
model.add(Dense(10))
model.add(Softmax())
model.summary()

In [ ]:
# Alternate way of model representation
model = Sequential([
                    Flatten(input_shape=(28,28)),
                    Dense(128, activation = "relu"),
                    Dense(128, activation = "relu"),
                    Dense(10, activation = "softmax")
                    ])
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_img, train_labels, epochs=10)

In [ ]:
print("Evaluate")
result = model.evaluate(test_img,test_labels, verbose=1)
dict(zip(model.metrics_names, result))

In [ ]:
# Predictions using the model
predictions = model.predict(test_img)
predictions


In [ ]:
# Getting the most probable class using argmax
probClass = np.argmax(predictions,axis = 1)
probClass


In [ ]:
# Take a random index
index = random.randint(0,10000)
print("Class of the product",test_labels[index])
# Extract the image from the test set for prediction
img = test_img[index:index+1,:,:]
print("Shape of the image",img.shape)

In [ ]:
# Plotting the selected image
plt.imshow(test_img[index,:,:])
plt.colorbar()
# Showing the image
plt.show()

In [ ]:
# predicting on the image and getting the class with the largest probability
print('Predicted class of the image :', np.argmax(model.predict(img)))

#### Building model with Functional API

In [ ]:
# Import the packages
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import Dense,ReLU,Activation,Flatten,Concatenate


In [ ]:
# Building the model architecture using Functional API
inputs = Input((28,28,))                  
x = Flatten()(inputs)                       
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dense(10)(x)
output = Activation('softmax')(x)
model = Model(inputs,output)     
model.summary()


In [ ]:
# A model architecture with multiple inputs and output
# first input network
input1 = Input((28,28))
x1 = Flatten()(input1)
x1 = Dense(128)(x1)
x1 = Activation('relu')(x1)
# second input network
input2 = Input((28,28))
x2 = Flatten()(input2)
x2 = Dense(128)(x2)
x2 = Activation('relu')(x2)
# merge input models
merge = concatenate([x1, x2])
# Network 1 output
hidden1 = Dense(64, activation='relu')(merge)
hidden2 = Dense(64, activation='relu')(hidden1)
output1 = Dense(10, activation='softmax')(hidden2)
# Network 2 output
hidden3 = Dense(64, activation='relu')(merge)
hidden4 = Dense(64, activation='relu')(hidden3)
output2 = Dense(10, activation='softmax')(hidden4)
# Constructing the Model object
model = Model(inputs=[input1, input2], outputs=[output1,output2])
# summarize layers
model.summary()


In [ ]:
# Compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'rmsprop' ,metrics=['accuracy'])
# Fitting data to the model
model.fit([train_img,train_img], train_labels, epochs=3)


#### Custom training using Gradient tape

In [ ]:
from tensorflow.keras.optimizers import Adam
import math

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# Building the model
model = Sequential()
model.add(Flatten(input_shape=(28,28),name="Flatten_layer"))
model.add(Dense(128, activation = "relu",name="First_activation_layer"))

model.add(Dense(10, activation = "softmax",name="Softmax_activation_layer"))
model.summary()

In [ ]:
# Hyperparameters
batch_size = 128 
epochs = 10
optimizer = Adam(learning_rate=0.001)

In [ ]:
#Building the step function 
def step(real_x, real_y):
    with tf.GradientTape() as tape:
        # Make prediction
        pred_y = model(real_x)
        # Calculate loss
        model_loss = tf.keras.losses.sparse_categorical_crossentropy(real_y, pred_y)
    # Calculate gradients
    model_gradients = tape.gradient(model_loss, model.trainable_variables)
    # Update model
    optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))
    return model_loss,model_gradients

In [ ]:
# Creating empty lists to store the values
gradientsList = []
lossList = []
modWeights = []
# Training loop
bat_per_epoch = math.floor(len(train_img) / batch_size)
# Storing the initial model weights
modWeights.append(model.get_weights())
for epoch in range(epochs):
    print('=', end='')
    for i in range(bat_per_epoch):
        # Get the batch counter
        n = i*batch_size
        # Get the model loss and gradients from the step function
        model_loss,model_gradients = step(train_img[n:n+batch_size], train_labels[n:n+batch_size])
        # Store the loss and gradients in the list
        lossList.append(model_loss)
        gradientsList.append(model_gradients)
    # Store the model weights in the list
    modWeights.append(model.get_weights())

In [ ]:
print("Total set of weights",len(modWeights))
print('Total number of parameters',len(modWeights[0]))


In [ ]:
print("Set of all losses",len(lossList))
print('Total number of gradient sets',len(gradientsList))


#### Topology with Dropout

In [ ]:
# Model topology definition
model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.summary()